# 07.03 Sentiment Analysis : NAVER MOVIE REVIEW

* https://github.com/e9t/nsmc

In [5]:
# 유니코드로 인코딩하며 읽기 위해 codecs 패키지를 사용 : 현재 데이터가 'UTF-8'로 인코딩되어있음. 파이썬의 경우, 데이터를 유니코드로 읽어야 함
# codecs 패키지는 데이터를 읽는 순간, 유니코드로 인코딩을 시켜주는 기능
# 일명 'streaming encoder'

import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]   # header 제외

In [13]:
from pprint import pprint

pprint(data[0])

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']


In [14]:
X = list(zip(*data))[1]
y = np.array(list(zip(*data))[2], dtype=int)

# 1. 전처리 : CountVectorizer, 모델 : NB-multinomial

In [35]:
# 전처리 : CountVectorizer ==>> 다항분포 샘플 생성
# 모델 : multinomialNB 
# 파이프라인으로 생성

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

In [36]:
# 모델 학습

model1.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [37]:
# 모델 성능 확인 (cross - validation 실시)

# 모델 성능 확인을 위한 test data 호출
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
    data_test = [line.split('\t') for line in f.read().splitlines()]
    data_test = data_test[1:]   # header 제외

In [38]:
# 모델 성능 확인을 위한 classifcation report 출력

X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.82     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [39]:
# 모델 예측 확인

# 부정적 단어 : 0 / 긍정적 단어 : 1

In [40]:
model1.predict(['꽝이야'])

array([0])

In [41]:
model1.predict(['짱이야'])

array([1])

In [42]:
model1.predict(['죽인다'])

array([1])

In [43]:
model1.predict(['우와'])

array([1])

In [44]:
model1.predict(['우왕'])

array([1])

In [45]:
model1.predict(['대박'])

array([1])

In [46]:
model1.predict(['연기'])

array([1])

In [47]:
model1.predict(['노잼'])

array([0])

In [48]:
model1.predict(['못한다'])

array([0])

# 2. 전처리 : TfidfVectorizer, 모델 : NB-multinomial

    - CountVectorizer 전처리 시, 성능 차이 크게 없음
   

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [50]:
model2.fit(X, y)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [51]:
print(classification_report(y_test, model2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.83     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



# 3. 전처리 : CountVectorizer, 모델 : NB-multinomial, 형태소 분석기 사용

    - 형태소 분석기 : konlpy - okt
   

In [52]:
from konlpy.tag import Okt
pos_tagger = Okt()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [53]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB()),
])

In [55]:
%%time
model3.fit(X, y)

CPU times: user 12min 59s, sys: 4.28 s, total: 13min 3s
Wall time: 12min 54s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_pos at 0x138e3bb90>,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [56]:
print(classification_report(y_test, model3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85     24827
           1       0.86      0.85      0.85     25173

    accuracy                           0.85     50000
   macro avg       0.85      0.85      0.85     50000
weighted avg       0.85      0.85      0.85     50000



# 4. 전처리 : CountVectorizer, 모델 : NB-multinomial, 형태소 분석기 사용(gram 수정)

    - 형태소 분석기 : konlpy - okt
    - gram : 1-2 gram 사용

In [57]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

In [58]:
%%time
model4.fit(X, y)

CPU times: user 16min 11s, sys: 5.78 s, total: 16min 17s
Wall time: 16min 7s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_pos at 0x138e3bb90>,
                                 use_idf=True, vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [59]:
print(classification_report(y_test, model4.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87     24827
           1       0.87      0.86      0.87     25173

    accuracy                           0.87     50000
   macro avg       0.87      0.87      0.87     50000
weighted avg       0.87      0.87      0.87     50000

